# Лабораторная работа №4

In [2]:
import scipy as sp
import numpy as np
from plotly import express as px

In [4]:
def Gauss(A, B):
    n = len(A)
    if n != len(B):
        raise ValueError("A and B must have the same length")
    # Triangle form
    for i in range(n):
        for j in range(i + 1, n):
            coeff = -(A[j][i] / A[i][i])
            for k in range(i, n):
                A[j][k] += coeff * A[i][k]
            B[j] += coeff * B[i]
    # Diagonal form
    for i in range(n - 1, -1, -1):
        for j in range(i - 1, -1, -1):
            coeff = -(A[j][i] / A[i][i])
            A[j][i] += coeff * A[i][i]
            B[j] += coeff * B[i]
    # Solution
    res = [B[i] / A[i][i] for i in range(n)]
    return res

def TridiagonalSolve(Mat, F):
    n = len(Mat)
    
    if any(len(row) != n for row in Mat):
        raise ValueError("Матрица должна быть квадратной")
    if len(F) != n:
        raise ValueError("Длина вектора F должна совпадать с размером матрицы")

    A = [0] * n
    B = [0] * n
    C = [0] * n
    
    for i in range(n):
        A[i] = Mat[i][i - 1] if i - 1 >= 0 else 0
        B[i] = Mat[i][i]
        C[i] = Mat[i][i + 1] if i + 1 < n else 0
    
    eps = [0] * (n - 1)
    eta = [0] * (n - 1)
    x = [0] * n
    
    eps[0] = -C[0] / B[0]
    eta[0] = F[0] / B[0]


    for i in range(1, n - 1):
        denominator = B[i] - A[i] * eps[i - 1]
        eps[i] = C[i] / denominator
        eta[i] = (F[i] + A[i] * eta[i - 1]) / denominator
                
    x[-1] = (F[-1] - A[-1] * eta[-1]) / (A[-1] * eps[-1] + B[-1])
    for i in range(n - 2, -1, -1):
        x[i] = eta[i] + eps[i] * x[i + 1]
    
    return x

In [5]:
c = 3 * 1e10 # Speed of light

class NanException(Exception):
    pass

class Configuration:
    def __init__(self):
        self.R = 0.35
        self.T0 = 8000
        self.Tw = 1800
        self.p =  2
        self.Imax = 1000
        self.Itmax = 80e-6
        self.tmax = 80e-6
        self.zmax = 1
        self.t0 = 0
        self.z0 = 0

        self.Tarray = [2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000]
        self.SigmaArray = [0.309e-3, 0.309e-2, 0.309e-1, 0.270, 0.205e+1, 0.606e+1, 0.120e+2, 0.199e+2, 0.296e+2, 0.411e+2, 0.541e+2]
        self.LambdaArray = [0.381e-3, 0.381e-3, 0.381e-3, 0.448e-3, 0.577e-3, 0.733e-3, 0.131e-2, 0.218e-2, 0.358e-2, 0.562e-2, 0.832e-2]
        self.cTArray = [1.90e-3, 1.90e-3, 0.95e-3, 0.75e-3, 0.64e-3, 0.61e-3, 0.66e-3, 0.66e-3, 1.15e-3, 1.79e-3, 2.02e-3]

        # For k interpolation
        self.TkArray = [2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
        self.KVariant = 2
        self.K1Array = [8.200E-03, 2.768E-02, 6.560E-02, 1.281E-01, 2.214E-01, 3.516E-01, 5.248E-01, 7.472E-01, 1.025E+00]
        self.K2Array = [1.6, 5.4, 1.280E+01, 2.500E+01, 4.320E+01, 6.860E+01, 1.024E+02, 1.458E+02, 2.000E+02]

        self.eps = 1e-4
    
    def I(self, t):
        return self.Imax/self.Itmax * t * np.exp(-(t / self.Itmax - 1))
    
    def Sigma(self, T):
        tarr = [np.log(x) for x in self.Tarray]
        sarr = [np.log(x) for x in self.SigmaArray]
        return np.exp(np.interp(np.log(T), tarr, sarr))    
        
    def Lambda(self, T):
        tarr = [np.log(x) for x in self.Tarray]
        larr = [np.log(x) for x in self.LambdaArray]
        return np.exp(np.interp(np.log(T), tarr, larr))
    
    def cT(self, T):
        tarr = [np.log(x) for x in self.Tarray]
        carr = [np.log(x) for x in self.cTArray]
        return np.exp(np.interp(np.log(T), tarr, carr))
    
    def k(self, T):
        tarr = [np.log(x) for x in self.TkArray]
        if self.KVariant == 1:
            karr = [np.log(x) for x in self.K1Array]
        else:
            karr = [np.log(x) for x in self.K2Array]
        return np.exp(np.interp(np.log(T), tarr, karr))
    
    def E(self, z: list[float], T : list[float], t: float):
        # sp.integrate.simpson(intFunc, z[:i+1])
        if len(z) != len(T):
            raise ValueError("z and T must have the same length")
        funcArr = [self.Sigma(T[i]) * z[i] for i in range(len(z))]
        integral = sp.integrate.simpson(funcArr, z)
        return self.I(t) / (2 * np.pi * self.R**2 * integral)
    
    def Tinit(self, z):
        return self.T0 + (self.Tw - self.T0) * (z)**self.p
    
    def Plank(self, T: float):
        return 3.084 * 1e-4 / (np.exp(4.799*1e4 / T) - 1)

    
    def q(self, z: list[float], u : list[float], T: list[float], zindex: int):
        if len(z) != len(u):
            raise ValueError("z and u must have the same length")
        if zindex >= len(z):
            raise ValueError("zindex must be less than len(z)")
        return c * self.k(T[zindex]) * (self.Plank(T[zindex]) - u[zindex])
    
    def Solveu(self, n, T):
        z = np.linspace(self.z0, self.zmax, n)
        h = np.diff(z)[0]
        
        k = lambda ind: self.k(T[ind])
        mat = [[0] * n for _ in range(n)]

        kappa = lambda ind: 2 / (k(ind) + k(ind + 1))
        mat[0][0] =  1/((k(0) + k(1)) * self.R)
        mat[0][1] = -mat[0][0]
        for i in range(1, n - 1):
            A = -(z[i] - h/2)*kappa(i - 1) / self.R**2 / h
            C = -(z[i] + h/2)*kappa(i) / self.R**2 / h
            B = (A + C - 3*k(i)*z[i]*h)
            mat[i][i - 1] = A
            mat[i][i] = B
            mat[i][i + 1] = C
        mat[-1][-2] = - (1/(k(-2) + k(-1))/self.R**2/h*(2 - h))
        mat[-1][-1] = 3*0.39 / self.R - mat[-1][-2]

        side = [0] * n
        for i in range(1, n - 1):
            side[i] = -3 * k(i) * z[i] * h * self.Plank(T[i])

        return TridiagonalSolve(mat, side)

    def Iteration(self, iterT:list[float], previousT: list[float], z: list[float], u: list[float], tau: float, t : float):
        '''
            iterT - T from previous iteration
            previousT - T from previous time step
        '''
        if len(z) != len(u):
            raise ValueError("z and u must have the same length")
        if len(z) != len(previousT):
            raise ValueError("z and previousT must have the same length")
        if len(z) != len(iterT):
            raise ValueError("z and iterT must have the same length")


        h = np.diff(z)[0]
        kappa = lambda ind: 2 * (self.Lambda(iterT[ind - 1]) * self.Lambda(iterT[ind])) / (self.Lambda(iterT[ind - 1]) + self.Lambda(iterT[ind]))
        mat = [[0] * len(z) for _ in range(len(z))]
        mat[0][0] = tau / self.R**2 * (self.Lambda(iterT[0]) * self.Lambda(iterT[1])) / (self.Lambda(iterT[0]) + self.Lambda(iterT[1]))
        mat[0][1] = -mat[0][0]
        F = [0] * len(z)
        F[0] = 0
        E = self.E(z, iterT, t)
        for i in range(1, len(z) - 1):
            A = tau / self.R**2 * (z[i] - h/2) * kappa(i) / h
            C = tau / self.R**2 * (z[i] + h/2) * kappa(i + 1) / h
            B = A + C + z[i]*h*self.cT(iterT[i])
            F[i] = z[i] * self.cT(iterT[i]) * previousT[i] * h + tau * h * z[i] * self.Sigma(iterT[i]) * E**2 - tau * h * self.q(z, u,  iterT, i) * z[i]
            mat[i][i - 1] = A
            mat[i][i] = B
            mat[i][i + 1] = C
        mat[-1][-2] = 0
        mat[-1][-1] = 1
        F[-1] = self.Tw            
        
        return TridiagonalSolve(mat, F)
    
    def SolveTimeStep(self, previousT: list[float], z: list[float], t : float, tau: float):
        if len(z) != len(previousT):
            raise ValueError("z and previousT must have the same length")

        u = self.Solveu(len(z), previousT)

        iterT = previousT
        cnt = 0
        while True:
            cnt += 1
            newT = self.Iteration(iterT, previousT, z, u, tau, t)
            if any(np.isnan(newT)):
                raise NanException("Nan in newT: iterT=", iterT, "previousT=", previousT, "z=", z, "u=", u, "tau=", tau, "t=", t)
            if all([abs(newT[i] - iterT[i])/newT[i] < self.eps for i in range(len(newT))]):
                break
            iterT = newT
            u = self.Solveu(len(z), iterT)
        
        return iterT, u
    
    def DiffEnergies(self, zArr, tArr, uArr, timeSlices):
        h = np.diff(zArr)[0]
        tau = np.diff(tArr)[0]

        t = tArr[-1]
        temp = timeSlices[-1]

        ElectricEnergy = self.I(t) * self.E(zArr, temp, t) / (2 * np.pi)


        EmissionEnergy = self.R**2 * sp.integrate.simpson([self.q(zArr, uArr, temp, i) * zArr[i] for i in range(len(zArr))], zArr)

        ConductionEnergy = - self.Lambda(temp[-1]) * (temp[-1] - temp[-2]) / h

        print(f"ElectricEnergy = {ElectricEnergy:.2e}, EmissionEnergy = {EmissionEnergy:.2e}, ConductionEnergy = {ConductionEnergy:.2e}")
        return abs(ElectricEnergy - EmissionEnergy - ConductionEnergy) / ElectricEnergy
    
    def Solve(self, zSteps, tSteps, verbose=True):
        zArr = np.linspace(self.z0, self.zmax, zSteps)
        tArr = np.linspace(self.t0, self.tmax, tSteps)

        tau = np.diff(tArr)[0]
        h = np.diff(zArr)[0]

        timeSlices = []
        timeSlices.append([self.Tinit(z) for z in zArr])
        uSlices = []
        uSlices.append(self.Solveu(zSteps, timeSlices[-1]))
        for t in tArr[1:]:
            temp, u = self.SolveTimeStep(timeSlices[-1], zArr, t, tau)
            if verbose:
                print(f"iter{len(timeSlices)} temp=", temp)
                print(f"iter{len(timeSlices)} u=", u)
            timeSlices.append(temp)
            uSlices.append(u)
        
        return zArr, tArr, uSlices, timeSlices
    
    def SolveUntilStationary(self, zSteps, tStep, statEps=1e-3, minSteps=100, verbose=True):
            zArr = np.linspace(self.z0, self.zmax, zSteps)
            tArr = [0]
            tau = tStep

            timeSlices = []
            timeSlices.append([self.Tinit(z) for z in zArr])
            uSlices = []
            uSlices.append(self.Solveu(zSteps, timeSlices[-1]))
            while True:
                tArr.append(tArr[-1] + tau)
                temp, u = self.SolveTimeStep(timeSlices[-1], zArr, tArr[-1], tau)
                if verbose:
                    print(f"iter{len(timeSlices)} temp0=", temp[0])
                timeSlices.append(temp)
                uSlices.append(u)
                if minSteps < len(timeSlices) and all([abs(timeSlices[-1][i] - timeSlices[-2][i])/timeSlices[-1][i] < statEps for i in range(len(timeSlices[-1]))]):
                    break

            return zArr, tArr, uSlices, timeSlices

    def SolveFindTimeStep(self, zSteps, initTSteps, tInc=100, verbose=True):
            tSteps = initTSteps
            while True:
                try:
                    zArr, tArr, uArr, timeSlices = self.Solve(zSteps, tSteps, verbose=verbose)
                    return zArr, tArr, uArr, timeSlices, tSteps
                except NanException:
                    tSteps += tInc
                    if verbose:
                        print(f"Nan in Solve: tSteps={tSteps}")

    def SolveFindUntilStationary(self, zSteps, initTau, tauMod=100, statEps=1e-3, minSteps=100, verbose=True):
            tau = initTau
            while True:
                try:
                    zArr, tArr, uArr, timeSlices = self.SolveUntilStationary(zSteps, tau, statEps, minSteps, verbose=verbose)
                    return zArr, tArr, uArr, timeSlices, tau
                except NanException:
                    tau /= tauMod
                    if verbose:
                        print(f"Nan in SolveUntilStationary: tau={tau}")

        
        


In [12]:
config = Configuration()

zSteps = 100
tSteps = 2000
config.tmax = 400e-6
config.Imax = 300

zArr, tArr, uArr, timeSlices = config.Solve(zSteps, tSteps, verbose=False)

fig = px.line()
for i in range(0, len(timeSlices), 50):
    fig.add_scatter(x=zArr, y=timeSlices[i])
fig.show()

# TempArr0 = [t[0] for t in timeSlices]
fig = px.line()
for i in range(0, len(zArr), 5):
    TempArr = [t[i] for t in timeSlices]
    fig.add_scatter(x=tArr, y=TempArr, name = f"z = {zArr[i]:.2f}")
fig.show()


In [16]:
config = Configuration()

zSteps = 20
config.I = lambda t: 100
config.KVariant = 1
initTau = 1e-6
diffEPS = 1e-2
statEps = 1e-4

while True:
    zArr, tArr, uArr, timeSlices, tau = config.SolveFindUntilStationary(zSteps, initTau, tauMod=2, statEps=statEps, minSteps=int(40e-6/initTau), verbose=False)
    print(f"tau = {tau}")
    print(f"zSteps = {zSteps}")
    diff = config.DiffEnergies(zArr, tArr, uArr[-1], timeSlices)
    print(f"diff = {diff}")
    if diff < diffEPS:
        break
    # initTau = tau / 2
    zSteps *= 2

TempArr0 = [t[0] for t in timeSlices]

fig = px.line(x=tArr, y=TempArr0)
fig.show()

tau = 1e-06
zSteps = 20
ElectricEnergy = 4.55e+02, EmissionEnergy = 4.28e+02, ConductionEnergy = 2.22e+01
diff = 0.012457928354363414
tau = 1e-06
zSteps = 40
ElectricEnergy = 4.55e+02, EmissionEnergy = 4.29e+02, ConductionEnergy = 2.40e+01
diff = 0.005996993385330239


In [ ]:
config = Configuration()

zSteps = 100
tSteps = 800
config.I = lambda t: 0
config.tmax = 400e-6 
config.eps = 1e-5

zArr, tArr, uArr, timeSlices, steps = config.SolveFindTimeStep(zSteps, tSteps, tInc=100, verbose=False)
print(f"steps={steps}")

fig = px.line()
for i in range(0, len(timeSlices), 50):
    fig.add_scatter(x=zArr, y=timeSlices[i])
fig.show()

TempArr0 = [t[0] for t in timeSlices]

fig = px.line(x=tArr, y=TempArr0)
fig.show()

steps=800


In [ ]:
TwValues = [1800, 3000, 5000, 7000, 8000]

for Tw in TwValues:
    config = Configuration()
    config.KVariant = 1
    config.Tw = Tw
    config.tmax = 200e-6
    zArr, tArr, uArr, timeSlices, steps = config.SolveFindTimeStep(zSteps=100, initTSteps=400, tInc=100, verbose=False)
    print(f"Tw={Tw}, steps={steps}")
    fig = px.line(x=tArr, y=[timeSlices[i][0] for i in range(len(timeSlices))])
    fig.update_layout(title=f"Tw={Tw}")
    fig.show()

    fig = px.line()
    for i in range(0, len(timeSlices), 50):
        fig.add_scatter(x=zArr, y=timeSlices[i])
    fig.update_layout(xaxis_title="z", yaxis_title="T")
    fig.update_layout(title=f"Tw={Tw}")
    fig.show()



Tw=1800, steps=400


Tw=3000, steps=400


Tw=5000, steps=400


Tw=7000, steps=400


Tw=8000, steps=400
